##### Project Title: Order Cleanup
###### Project Purpose: Data Cleaning for Easy Ingestion
###### Scenario: Customers Inputing Their Orders. (We all know how inputs from uses are not standardized)

In [1]:
# The main libs for this project
import numpy as np 
import pandas as pd

In [2]:
# The data is on my local machine
Orders = pd.read_excel('Mock Customer Orders.xlsx')
# Filling null values with blank. This will help me when slicing and dicing columns and rows
Orders = Orders.fillna('')
Orders.head(5)

,Company Name,Item Number,Notes,Submitted Date
0,Twinder,994608755-3,1G6KF549X5U998061,2021-11-15
1,Digitube,665846669-6,1GYEC63898R798565,2022-05-15
2,Plajo,801069542-4/23/45/65/6,3LNDL2L33BR788927,2022-02-09
3,Skynoodle,"737638507, W4325T,567U",WAU4GAFB5BN183376,2022-04-29
4,Vipe,353800830-2,WBABN534X2P304580,2022-05-06


In [3]:
# Spliting the rows that have more than 1 item and are presented with a , delimeter
Orders['Item Number'] = Orders['Item Number'].str.split(',')
# Exploding so that they I can create new rows with the slip values
Orders = Orders.explode('Item Number').reset_index(drop=True)
cols = list(Orders.columns)
cols.append(cols.pop(cols.index('Notes')))
Orders = Orders[cols]
# Display the changes. Mainly checking to see if the split (,) worked.
Orders.head(10)

,Company Name,Item Number,Submitted Date,Notes
0,Twinder,994608755-3,2021-11-15,1G6KF549X5U998061
1,Digitube,665846669-6,2022-05-15,1GYEC63898R798565
2,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927
3,Skynoodle,737638507,2022-04-29,WAU4GAFB5BN183376
4,Skynoodle,W4325T,2022-04-29,WAU4GAFB5BN183376
5,Skynoodle,567U,2022-04-29,WAU4GAFB5BN183376
6,Vipe,353800830-2,2022-05-06,WBABN534X2P304580
7,Kazio,737638507,2022-05-26,WA1CMAFE3ED565640
8,Kazio,W4325T,2022-05-26,WA1CMAFE3ED565640
9,Kazio,567U,2022-05-26,WA1CMAFE3ED565640


In [4]:
# Spliting the - delimeter and keep the part before the delimeter
Orders['Item Number1'] =pd.Series(Orders['Item Number'].str.split('-'[0]).str[0])
# Keep the second part after the - delimeter
Orders['Item Number2'] =pd.Series(Orders['Item Number'].str.split('-'[0]).str[1])
# Spliting by delimeter / and explode to populate new rows with the split data
Orders['Item Number3']= pd.Series(Orders['Item Number2'].str.split('/'))
Orders = Orders.explode('Item Number3').reset_index(drop=True)
Orders = Orders.fillna('')
# Taking care of empty rows/ valies
Orders = Orders.where(Orders['Item Number1'].notnull())
Orders.head(10)


,Company Name,Item Number,Submitted Date,Notes,Item Number1,Item Number2,Item Number3
0,Twinder,994608755-3,2021-11-15,1G6KF549X5U998061,994608755,3,3
1,Digitube,665846669-6,2022-05-15,1GYEC63898R798565,665846669,6,6
2,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,4
3,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,23
4,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,45
5,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,65
6,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,6
7,Skynoodle,737638507,2022-04-29,WAU4GAFB5BN183376,737638507,,
8,Skynoodle,W4325T,2022-04-29,WAU4GAFB5BN183376,W4325T,,
9,Skynoodle,567U,2022-04-29,WAU4GAFB5BN183376,567U,,


In [5]:
# Concat the new colums with a condition
Orders['Item#'] = np.where(Orders['Item Number2'] =='', Orders['Item Number1'], Orders['Item Number1'] + '-'+ Orders['Item Number3'])
# Selecting the updated df and droping rows that have nothing
Orders = Orders.where(Orders['Item Number']!= '')
Orders= Orders.dropna(axis =0)
Orders.head(10)

,Company Name,Item Number,Submitted Date,Notes,Item Number1,Item Number2,Item Number3,Item#
0,Twinder,994608755-3,2021-11-15,1G6KF549X5U998061,994608755,3,3,994608755-3
1,Digitube,665846669-6,2022-05-15,1GYEC63898R798565,665846669,6,6,665846669-6
2,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,4,801069542-4
3,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,23,801069542-23
4,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,45,801069542-45
5,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,65,801069542-65
6,Plajo,801069542-4/23/45/65/6,2022-02-09,3LNDL2L33BR788927,801069542,4/23/45/65/6,6,801069542-6
7,Skynoodle,737638507,2022-04-29,WAU4GAFB5BN183376,737638507,,,737638507
8,Skynoodle,W4325T,2022-04-29,WAU4GAFB5BN183376,W4325T,,,W4325T
9,Skynoodle,567U,2022-04-29,WAU4GAFB5BN183376,567U,,,567U


In [6]:
# Clean data for ingestion or more analysis
Orders = Orders[['Company Name','Item#', 'Submitted Date', 'Notes']]
Orders.head(10)

,Company Name,Item#,Submitted Date,Notes
0,Twinder,994608755-3,2021-11-15,1G6KF549X5U998061
1,Digitube,665846669-6,2022-05-15,1GYEC63898R798565
2,Plajo,801069542-4,2022-02-09,3LNDL2L33BR788927
3,Plajo,801069542-23,2022-02-09,3LNDL2L33BR788927
4,Plajo,801069542-45,2022-02-09,3LNDL2L33BR788927
5,Plajo,801069542-65,2022-02-09,3LNDL2L33BR788927
6,Plajo,801069542-6,2022-02-09,3LNDL2L33BR788927
7,Skynoodle,737638507,2022-04-29,WAU4GAFB5BN183376
8,Skynoodle,W4325T,2022-04-29,WAU4GAFB5BN183376
9,Skynoodle,567U,2022-04-29,WAU4GAFB5BN183376
